In [ ]:
console.log("Hello, world!")

This tutorial goes through a couple of examples that show the evolution of E-ID solutions.

We will go through the following examples 

1. Typical message signing without privacy
2. Adding Selective Disclosure
3. Adding Zero-Knowledge proofs
4. Adding Unlinkability

In [ ]:
// We start by creating a typical E-ID object that we will use through out this tutorial

const EID_DATA = {
    name: "Ahmed",
    age: "22",
    profession: "RSE"
}

## A typical message signature in today's world

In [ ]:
// Sender 
import * as crypto from 'crypto';

// Creating Key Pair
const { publicKey, privateKey } = crypto.generateKeyPairSync('rsa', {
    modulusLength: 4096,
    publicKeyEncoding: {
      type: 'spki',   

      format: 'pem'
    },
    privateKeyEncoding: {
      type: 'pkcs8',
      format: 'pem',
    }
  });

const message = JSON.stringify(EID_DATA)

// sign the secret data 
const signer = crypto.createSign('SHA256');
signer.update(message);
const signature = signer.sign(privateKey, 'base64')

Holder maintains a single signature at all times.  
Everytime it is presented, it's the same!

In [ ]:
// Receiver - receives message, signature, and publicKey

let verifier = crypto.createVerify('SHA256');
verifier.update(message)
console.log(verifier.verify(publicKey, signature, 'base64'))

// try another false message
verifier = crypto.createVerify('SHA256');
verifier.update("Another random message!")
console.log(verifier.verify(publicKey, signature, 'base64'))

Right now, we need to send all of the EID_DATA content to be able to verify it. 

What if we want to send just the name, and verify that? 

RSA signatures are holistic; you either verify the entire signed message or nothing at all.

To do this, an alternative method should we used. In the following sections we will use a modern crypto scheme called BBS+

# Now with Selective Disclosure

# Now with BBS+

In [ ]:
import { BBS } from './src/bbs';

In [ ]:
const bbs = new BBS();

// generate key pair
const privateKey = bbs.KeyGen(crypto.randomBytes(32));
const publicKey = bbs.SkToPk(privateKey);

In [ ]:
// For the issuer

// preparing the data
const messagesToSign = Object.values(EID_DATA)

// we hash the messages (One-way process)
let hashedMessages = messagesToSign.map(message => bbs.MapMessageToScalarAsHash(Buffer.from(message)));

// signing the thing
const generators = await bbs.create_generators(messagesToSign.length);
const signature = bbs.Sign(privateKey, publicKey, Buffer.from("HEADER", "utf-8"), hashedMessages, generators)

// let's verify these hashes before we send things out 
bbs.Verify(publicKey, signature, Buffer.from("HEADER", "utf-8"), hashedMessages, generators);

// Issuer sends messagesToSign, signature, and hashedMessages to Holder

In [ ]:
signature

Both the EID_DATA and signature are transmitted to the EID holder.

Then, we assume the holder/ owner of the EID wants to only share name, and profession with a certain entity. 

## Holder

In [ ]:

// holder determines that they want to disclose only the first and last messages (name, profession)
const IndexesOfFieldsToDisclose = [0,2]

// const HolderGenerators = await bbs.create_generators(messagesToSign.length);

// holder generates a proof
const proof = bbs.ProofGen(
    publicKey, signature, 
    Buffer.from("HEADER", "utf-8"), Buffer.from("PRESENTATION HEADER", "utf-8"),
    hashedMessages, generators, IndexesOfFieldsToDisclose
)
const MessagesToDisclose = messagesToSign.filter((val, i) => {return IndexesOfFieldsToDisclose.includes(i)})

MessagesToDisclose

In [ ]:
// Verifier - received publicKey, proof, disclosed data, disclosed data indexes, all hashed messages

const verifierGenerators = await bbs.create_generators(messagesToSign.length);
let verifierHashedMessages = MessagesToDisclose.map(message => bbs.MapMessageToScalarAsHash(Buffer.from(message)));

bbs.ProofVerify(
    publicKey, proof,
    Buffer.from("HEADER", "utf-8"), Buffer.from("PRESENTATION HEADER", "utf-8"),
    verifierHashedMessages, verifierGenerators, IndexesOfFieldsToDisclose
);

MessagesToDisclose

In [ ]:
// Testing with fake data
const fakeMessages = ['Ahmed', 'CEO']

const verifierGenerators = await bbs.create_generators(messagesToSign.length);

// we use fakeMessages instead of MessagesToDisclose here
let verifierHashedMessages = fakeMessages.map(message => bbs.MapMessageToScalarAsHash(Buffer.from(message)));

bbs.ProofVerify(
    publicKey, proof,
    Buffer.from("HEADER", "utf-8"), Buffer.from("PRESENTATION HEADER", "utf-8"),
    verifierHashedMessages, verifierGenerators, IndexesOfFieldsToDisclose
);

MessagesToDisclose

In [ ]:
// Unlinkability 

const customGenerators = await bbs.create_generators(messagesToSign.length);

const proof1 = bbs.ProofGen(
    publicKey, signature, 
    Buffer.from("HEADER", "utf-8"), Buffer.from("PRESENTATION HEADER", "utf-8"),
    hashedMessages, customGenerators, IndexesOfFieldsToDisclose
)

const proof2 = bbs.ProofGen(
    publicKey, signature, 
    Buffer.from("HEADER", "utf-8"), Buffer.from("PRESENTATION HEADER", "utf-8"),
    hashedMessages, customGenerators, IndexesOfFieldsToDisclose
)

proof1 == proof2

In [ ]:
new Buffer(proof2).toString("hex")

In [ ]:
new Buffer(proof1).toString("hex")